## Compute Phase Encoding

In [ ]:
from os import path,makedirs
from os.path import exists
import numpy as np
import matplotlib.pyplot as plt
from brainpipe.system import study
from brainpipe.feature import power, amplitude, sigfilt, phase, PLF
from brainpipe.visual import *
from brainpipe.statistics import *
from itertools import product

## Compute phase 
    Instantaneous phase computation, value for the middle of each time window

In [ ]:
from brainpipe.feat.utils._feat import _manageWindow
st = study('Olfacto')
subjects = ['FERJ','MICP','VACJ','SEMC','LEFC','PIRJ','CHAF']#
conds = {'subj':('LowHigh',['low', 'high'])}

for su, cond in product(subjects[:], sorted(conds)):
    pathdata = path.join(st.path,'database/Encoding_EpiPerf_'+conds[cond][0]+'/')
    path2save = path.join(st.path,'feature/1_Phase_Encoding_EpiPerf_'+conds[cond][0]+'/')
    # Define phase settings 
    f = [[0.1,1.5],[2,3],[3,7],[8,13]]
    width, step = None,None
    fname = ['VLFC','delta','theta', 'alpha']
    
    for c in conds[cond][1]:
        # Load file :
        fi = su+'_odor_'+c+'_bipo_sel_physFT.npz'
        loadname = pathdata+fi
        if path.isfile(loadname):
            mat = np.load(loadname)
            print(mat.files)
            x = mat['x']
            n_elec, npts, n_trials = x.shape
            sf = 512.
            print ('--> compute phase on : ', fi, 'data', x.shape, 'npts', npts, 'ntrials',n_trials, 'nelec', n_elec)              

            # Define and save phase objects :
            phase0bj = phase(sf, npts=npts, f=f, method='hilbert', width=width, step=step)
            kwargs = {}
            kwargs['f'], kwargs['fname'] = f, fname
            kwargs['labels'], kwargs['channels'], kwargs['xyz'] = mat['labels'], mat['channels'], mat['xyz']
            kwargs['s_aal'], kwargs['s_BA'] = mat['aal'], mat['BA']
            kwargs['Mai_RL'], kwargs['Mai']  = mat['Mai_RL'], mat['Mai']
            #print('time vector',time)
            xPhase,  kwargs['phase_pval']= phase0bj.get(x,n_jobs=5)
            # Select middle instantaneous phase for each window
            sWidth, sStep = 358, 51
            wins, sTime = _manageWindow(x.shape[1], width=sWidth, step=sStep)
            kwargs['time'] = (np.array(sTime)-3*512)/512
            print('stime', (np.array(sTime)-3*512)/512)
            idx = [int(i) for i in sTime]  
            sPhase = np.array([])
            for i in idx:
                sPhase = np.concatenate((sPhase,xPhase[:,:,i,np.newaxis,:]), axis = 2) if np.size(sPhase) else xPhase[:,:,i,np.newaxis,:]
            kwargs['xPhase'], kwargs['sPhase'] = xPhase, sPhase
            save_phase = path2save+fi.replace('.npz', '_phase.npz')
            np.savez(save_phase, **kwargs)
            del kwargs['xPhase'],  kwargs['phase_pval'], kwargs['fname']
            del kwargs, x, sf, n_elec, n_trials

### Plot Single-trial low frequency phase over time
    by frequency bins (VLFC, delta, theta, alpha)

In [ ]:
import matplotlib.colors as colors
from os.path import exists
from os import makedirs
st = study('Olfacto')
subjects = ['FERJ','MICP','VACJ','SEMC','LEFC','PIRJ','CHAF']
conds = {'subj':('LowHigh',['low', 'high'])}
f = [[0.1,1.5],[2,3],[3,7],[8,13],[13,30]]
fname = ['VLFC','delta','theta', 'alpha']

for su, cond in product(subjects[:], sorted(conds)):
    pathdata = path.join(st.path,'feature/1_Phase_Encoding_EpiPerf_'+conds[cond][0]+'/')
    for c in conds[cond][1]:
        # Load file :
        fi = su+'_odor_'+c+'_bipo_sel_physFT_phase.npz'
        loadname = pathdata+fi
        if path.isfile(loadname):
            mat = np.load(loadname)
            x, time = mat['sPhase'], mat['time'] #nelecs, nfreq, nwins,ntrials
            labels = mat['Mai_RL']
            x = np.rad2deg(x)
            n_elec, nfreq, npts, n_trials = x.shape
            print ('--> compute phase on : ', fi, 'data', x.shape, 'npts', npts, 'ntrials',n_trials, 'nelec', n_elec)              
            #Plot single-trial phase over time
            for f, elec in product(range(nfreq),range(n_elec)):
                path2save = pathdata+'Single_trial/'+fname[f]+'/'
                if not exists(path2save):
                    makedirs(path2save)
                phase_plot = x[elec,f,17:52,:].swapaxes(0,1)
                plt.imshow(phase_plot, cmap='rainbow',norm=colors.Normalize(vmin=-180,vmax=180),
                   interpolation='none', origin ='lower', aspect='auto', extent=[-1,2.5,0,n_trials])
                plt.title('Trial by trial '+fname[f]+' phase modulation - '+c.capitalize()+
                          ' Memory \n'+ labels[elec])
                plt.ylabel('Trials')
                plt.xlabel('Time (s)')
                cb = plt.colorbar(ticks=[-180, -90, 0, 90,180], label='Phase (°)')
                filename = su+'_ST_phase_'+fname[f]+'_'+str(elec)+'_'+labels[elec]+'_'+c+'.png'
                plt.savefig(path2save+filename)
                plt.clf()
                plt.close()

### Check all phase files dimensions

In [ ]:
st = study('Olfacto')
files = st.search('_sel_phys_phase_test.npz', folder='feature/1_Phase_Encoding_EpiPerf_4500_expi_noart/')
for fi in files:
    loadname = path.join(st.path, 'feature/1_Phase_Encoding_EpiPerf_4500_expi_noart/', fi)
    mat = np.load(loadname)
    x = np.load(loadname)['phase']
    # nfreq, nelec, nwin, ntrials
    print (fi, x.shape, x[0,2,:,0])
    print(mat.files)


### Compute & Plot PLF

In [ ]:
from brainpipe.feat.utils._feat import _manageWindow

st = study('Olfacto')
subjects = ['FERJ','MICP','VACJ','SEMC','LEFC','PIRJ','CHAF']
conds = {'subj':('LowHigh',['low', 'high'])}

for su, cond in product(subjects[:], sorted(conds)):
    pathdata = path.join(st.path,'database/Encoding_EpiPerf_'+conds[cond][0]+'/')
    path2save = path.join(st.path,'feature/1_Phase_Encoding_EpiPerf_'+conds[cond][0]+'/PLF_plots/')
    if not exists(path2save):
        makedirs(path2save)
    # Define phase settings 
    f = (0.1,13,0.5, 0.2)
    #width, step = 358,51
    width, step = None,None
    
    for c in conds[cond][1]:
        # Load file :
        fi = su+'_odor_'+c+'_bipo_sel_physFT.npz'
        loadname = pathdata+fi
        if path.isfile(loadname):
            mat = np.load(loadname)
            print(mat.files)
            x = mat['x']
            labels = mat['Mai_RL']
            n_elec, npts, n_trials = x.shape
            sf = 512.
            print ('--> compute PLF on : ', fi, 'data', x.shape, 'npts', npts, 'ntrials',n_trials, 'nelec', n_elec)              
            
            for elec in range(n_elec):
                # Define and save power objects :
                n_elec, npts, n_trials = x.shape
                print ('--> compute PLF on : ', elec,'/', n_elec)  
                PLFObj = PLF(sf, npts=npts, f=f, method='hilbert1', width=width, step=step,
                            filtname='fir1',axis=0,cycle=3)
                PLF_plot, pvals = PLFObj.get(x[elec,:,:],n_jobs=5)
                print(PLF_plot.shape)
                #Time vector
                npts = len(range(1024,2816))
                time = np.arange(-512,npts-512,52)/512

                # Plot PLF for one electrode
                fig = plt.figure(figsize=(5, 4))
                title = su+' Phase locking factor - '+c.capitalize()+' Memory \n'+ labels[elec]+ ' elec('+str(elec)+')'
                fig = PLFObj.plot2D(fig, PLF_plot[:,:,1024:2816], cmap='viridis', xvec=time,
                         yvec=PLFObj.yvec, xlabel='Time (s)', vmin=0, vmax=0.7, ycb=-8,
                         ylabel='Frequency (hz)', figtitle=title, cblabel='PLF', 
                        pltype='imshow', resample=(0.1, 0.1),subspace={'top':0.85},
                        contour={'data':pvals, 'level':[0.001,], 'colors':['w'], 'linewidths':[1]},)
                filename = su+'_PLF_'+str(elec)+'_'+labels[elec]+'_'+c+'.png'
                plt.savefig(path2save+filename)
                plt.clf()
                plt.close()
            del PLF_plot, 
            del x, sf, n_elec, n_trials